In [61]:
import math
import numpy as np
import pandas as pd
import scipy
import scipy.signal as sg

import requests
import time
from datetime import datetime, timedelta

import matplotlib.pyplot as plt


In [62]:

def fetch_historical_data(base_curr, quote_curr, start_time, end_time, granularity=86400):
    """
    Fetches all available historical OHLCV data from Coinbase API.
    
    Args:
    - base_curr (str): Cryptocurrency base (e.g., "BTC", "ETH").
    - quote_curr (str): Cryptocurrency base (e.g., "USDT").
    - start_time (str): Start date in "YYYY-MM-DD" format.
    - end_time (str): End date in "YYYY-MM-DD" format.
    - granularity (int): Timeframe in seconds (default: 86400 for daily data).
    
    Returns:
    - pd.DataFrame: Dataframe containing all historical OHLCV data.
    """

    # Convert symbol format (BTCUSDT -> BTC-USD)
    pair = f"{base_curr}-{quote_curr}"

    url = f"https://api.exchange.coinbase.com/products/{pair}/candles"
    
    # Convert start_time and end_time to datetime objects
    start_dt = datetime.strptime(start_time, "%Y-%m-%d")
    end_dt = datetime.strptime(end_time, "%Y-%m-%d")

    all_data = []
    
    while start_dt < end_dt:
        batch_end = min(end_dt, start_dt + timedelta(seconds=granularity * 300))
        
        params = {
            "granularity": granularity,
            "start": start_dt.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "end": batch_end.strftime("%Y-%m-%dT%H:%M:%SZ")
        }

        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code}, {response.text}")
            break

        data = response.json()
        
        if not data:
            print("No data found for this period.")
            break
        
        all_data.extend(data)
        start_dt = batch_end + timedelta(seconds=granularity)

    if not all_data:
        print("No historical data available.")
        return None

    # Convert to DataFrame
    df = pd.DataFrame(all_data, columns=["timestamp", "low", "high", "open", "close", "volume"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df = df.sort_values("timestamp").reset_index(drop=True)

    return df

# Example Usage
df = fetch_historical_data("btc", "usdt", "2025-01-01", "2025-03-15")
print(df.head())


   timestamp       low      high      open     close      volume
0 2025-01-01  92850.82  95152.46  93551.63  94595.69  419.531431
1 2025-01-02  94384.72  97849.19  94604.24  97023.09  449.988917
2 2025-01-03  96105.41  98985.11  97028.27  98180.77  304.594067
3 2025-01-04  97536.31  98777.00  98265.11  98250.13   54.313027
4 2025-01-05  97269.93  98842.08  98265.91  98426.14   58.142619


In [63]:
df.close

0     94595.69
1     97023.09
2     98180.77
3     98250.13
4     98426.14
        ...   
69    82964.97
70    83682.95
71    81082.78
72    83981.73
73    84357.01
Name: close, Length: 74, dtype: float64

## Indicators 

In [ ]:

def sma(price, n):
  return price.rolling(n).mean()
def wma(price, n):
  return price.ewm(com=n).mean()

def highpass(Data, n=48):
  a	= (0.707*2*math.pi) / n

  alpha1 = (math.cos(a)+math.sin(a)-1)/math.cos(a);
  b	= 1-alpha1/2
  c	= 1-alpha1

  ret = [0] * len(Data)
  for i in range(2, len(Data)):
    ret[i] = b*b*(Data.iloc[i]-2*Data[i-1]+Data.iloc[i-2])+2*c*ret[i-1]-c*c*ret[i-2]

  return pd.Series(ret, index=Data.index)

def lowpass(Data,n):
  a = 2.0/(1+n)

  lp = [Data[0], Data[1]] + [0] * (len(Data) - 2)
  for i in range(2, len(Data)):
    lp[i] = (a-0.25*a*a)*Data[i]+ 0.5*a*a*Data[i-1]\
      - (a-0.75*a*a)*Data[i-2]\
      + 2*(1.-a)*lp[i-1]\
      - (1.-a)*(1.-a)*lp[i-2]

  return pd.Series(lp, index=Data.index)

def hullma(price, n):
  wma1 = wma(price, n//2)
  wma2 = wma(price, n)
  return wma(wma1 * 2 - wma2, int(math.sqrt(n)))

def zlma(price, n):
  """
  John Ehlers' Zero lag (exponential) moving average
  https://en.wikipedia.org/wiki/Zero_lag_exponential_moving_average
  """
  lag = (n - 1) // 2
  series = 2 * price - price.shift(lag)
  return wma(series, n)


def detrend(price, n):
  return price - highpass(price, n)


trends = {
    'sma': sma,
    'wma': wma,
    'lowpass': lowpass,
    'hullma': hullma,
    'zlma': zlma,
    'detrend': detrend,
}

In [66]:
ma_20 = df.close.rolling(20).mean()
ma_60 = df.close.rolling(60).mean()